### Régression Ridge

### Problématique

Analyser Comment les indicateurs économiques de différents pays, telles que le revenu, les dépenses et les dépenses par fonction ect, influencent-elles le PIB des pays? 

ET comment la régularisation par régression Ridge peut-elle faciliter l'identification de relations significatives tout en améliorant la précision des estimations?

#### Étape 1: Importation des bibliothèques necessaires

In [284]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

#### Étape 2: Chargement de données

In [191]:

data = pd.read_csv('dataset_fin.csv')
df = pd.DataFrame(data)

In [192]:
data.head()

,Unnamed: 0,Reference area,GDP,TIME_PERIOD,Income,Expenditure,Components of aggregate demand,Production,Households,Government,Government expenditure by fun,Corporations,Capital,Economic_Classification
0,0,Australia,3.441992,2022,2.630387,6.318369,3.090439,3.665822,39871.843115,37.198877,37.198877,2.826235,102.959793,Emerging
1,1,Austria,5.277894,2022,0.676556,4.911191,2.438285,6.070283,40299.683652,53.010387,53.010387,3.702775,102.748410,Emerging
2,2,Belgium,4.233432,2022,0.945041,3.635884,1.815193,4.515443,36499.838230,52.240785,52.240785,5.092422,102.937581,Developing
3,3,Canada,4.189036,2022,5.478162,5.464745,2.960486,4.139944,34203.373443,41.238013,41.238013,6.555331,103.696381,Emerging
4,4,Chile,2.058740,2022,-1.181835,1.642339,1.019797,2.282649,16552.007381,27.749244,27.749244,4.490536,103.703984,Developing


#### Etape3: Sélection des Variables

In [200]:
# Variables explicatives
X = data[['Income', 'Components of aggregate demand', 'Expenditure', 'Production', 'Households', 'Government', 'Corporations', 'Capital','Government expenditure by fun']]

# Variable cible
y = data['GDP']

#### Étape 4: Évaluation de la Multicolinéarité avec VIF

Le rôle de l'évaluation de la multicolinéarité à travers le calcul du facteur d'inflation de la variance (VIF), est crucial pour répondre à la problématique de l'analyse de l'impact des différents variables explicatives sur le PIB. Voici comment cela se manifeste :

**Identification des Relations entre Variables** :

Le VIF permet d'identifier si les variables explicatives sont corrélées entre elles.Géneralement Une forte multicolinéarité peut rendre difficile la détermination de l'effet individuel de chaque variable sur la variable cible (le PIB).


In [203]:
# Évaluation de la multicolinéarité avec VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_results = calculate_vif(X)
print("VIF Results:")
print(vif_results)

VIF Results:
                         Variable        VIF
0                          Income   1.405750
1  Components of aggregate demand  25.899816
2                     Expenditure  33.260355
3                      Production   7.937708
4                      Households  18.212370
5                      Government        inf
6                    Corporations   6.984738
7                         Capital  35.080917
8   Government expenditure by fun        inf


C:\Users\utilisateur\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


**Interprétation des Résultats du VIF**

 **VIF< 5**:
 
 -Les résultats du VIF montrent que certaines variables, comme **Income**, présentent une faible multicolinéarité ((VIF = 1.405750) < 5), ce qui permet d'évaluer son impact sur le PIB de manière fiable.


 **5<VIF<10** :
 
D'autres, comme **Production** (VIF = 7.937708)et **Corporations**(VIF = 6.984738)ont une multicolinéarité modérée (5 < VIF < 10) L'impact sur le PIB peut être évalué mais nécessitant une certaine prudence dans l’interprétation. 




 **VIF > 10**:
 
En revanche, des variables telles que **Components of Aggregate Demand (VIF = 25.899816)**,**Expenditure (VIF = 33.260355)**,**Households (VIF = 18.212370)**et**Capital (VIF = 35.080917)**  affichent un VIF élevé (> 10), indiquantMulticolinéarité élevée. Ces variables sont fortement corrélée avec d'autres, rendant difficile l'isolement de l'impact individuel de chacun sur le PIB



**VIF = inf**:

 Enfin, des valeurs infinies (VIF = inf) pour certaines variables, comme **Government** et **Government Expenditure by Fun**, révèlent une colinéarité parfaite, ce qui peut rendre difficile l'estimation précise de leur impact sur le PIB. 
 


#### Étape 5: Séparation des Données

Cette étape divise les données en ensembles d'entraînement et de test, permettant d'évaluer les performances des modèles sur des données non vues.

In [206]:
# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#### Étape 6: Normalisation des données

Dans notre analyse , les variables peuvent avoir des unités et des plages de valeurs très différentes (par exemple, le revenu en milliers et les dépenses en millions). Standardiser les donnes  permet d'éviter que certaines variables dominent l'analyse simplement à cause de leur échelle.

In [209]:
# Standardisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Étape 7: Régression Classique

Dans le cadre de notre problématique, la régression classique fournit une première approche pour analyser l'impact de chaque variable explicative  sur le PIB.

In [212]:

# Régression classique
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

#### Étape 8: Affichage des Coefficients de la Régression Linéaire

In [289]:

# Afficher les coefficients de la régression linéaire
coefficients_linear = pd.DataFrame(linear_model.coef_, X.columns, columns=['Coefficient (Linear)'])
print("Coefficients de la Régression Linéaire:")
print(coefficients_linear)

Coefficients de la Régression Linéaire:
                                Coefficient (Linear)
Income                                      0.099650
Components of aggregate demand              0.262501
Expenditure                                -0.053378
Production                                  2.095101
Households                                 -0.328577
Government                                  0.046905
Corporations                                0.135829
Capital                                     0.109041
Government expenditure by fun               0.046905


Les coefficients de la régression linéaire fournissent des informations cruciales sur l'impact relatif de chaque variable explicative sur le PIB. Voici une interprétation de chaque coefficient et son importance :

**Income (Coefficient = 0.099650)** :
Interprétation : Une augmentation d'une unité dans le revenu est associée à une augmentation de 0.09965 unités du PIB. Cela indique un impact positif et significatif du revenu sur le PIB.

**Components of Aggregate Demand (Coefficient = 0.262501)** :
Interprétation : Cette variable a un coefficient positif élevé, suggérant qu'une augmentation des composants de la demande agrégée entraîne une augmentation substantielle du PIB. Cela souligne l'importance de la demande agrégée dans la croissance économique.

**Production (Coefficient = 2.095101)** :
Interprétation : Ce coefficient très élevé montre que la production a un impact majeur sur le PIB. Une augmentation d'une unité de production entraîne une augmentation de plus de 2 unités du PIB, soulignant son rôle central dans l'économie.

En somme, bien que les revenus , la composante de demende aggrée et  la production ont un impact beaucoup plus marqué, toutes les variables notament (les dépenses gouvernementales,les corporations et l'investissement en capital) ont des effets positifs et significatifs sur le PIB ,chacune à des degrés différents.


**Households (Coefficient = -0.328577)** :
Interprétation : Un coefficient négatif indique que les variations liées aux ménages ont un impact négatif sur le PIB. Cela pourrait suggérer que des dépenses de ménages peuvent nuire à la croissance économique.

**Expenditure (Coefficient = -0.053378)** :
Interprétation : Un coefficient négatif indique qu'une augmentation des dépenses est associée à une légère diminution du PIB. Cela peut sembler contre-intuitif et nécessite une analyse plus approfondie pour comprendre cette dynamique.



#### Étape 9: Régression ridge

La régression Ridge est essentielle pour obtenir des résultats plus fiables, surtout lorsque des corrélations élevées entre les variables peuvent fausser les résultats.

In [221]:

# Régression Ridge
ridge_model = Ridge(alpha=1.0)  # Ajustez alpha selon vos besoins
ridge_model.fit(X_train_scaled, y_train)


Ridge()

#### Étape 10: Affichage des Coefficients de la Régression Ridge

In [227]:

# Coefficients de la régression Ridge
coefficients_ridge = pd.DataFrame(ridge_model.coef_, X.columns, columns=['Coefficient'])
print("Coefficients (Régression Ridge):")
print(coefficients_ridge)


Coefficients (Régression Ridge):
                                Coefficient
Income                             0.093455
Components of aggregate demand     0.192340
Expenditure                        0.064755
Production                         1.994780
Households                        -0.313627
Government                         0.051668
Corporations                       0.114302
Capital                            0.125254
Government expenditure by fun      0.051668


Les coefficients montrent comment chaque variable influence le PIB.

**Income (Revenu)** : 0.0935
Pour chaque unité d'augmentation de revenu, le PIB augmente en moyenne de 0.0935 unités.

**Components of Aggregate Demand (Composantes de la Demande Agrégée)** : 0.1923
Une augmentation d'une unité dans les composantes de la demande agrégée est associée à une augmentation du PIB de 0.1923 unités.

**Production** : 1.9948
Pour chaque unité d'augmentation de la production, le PIB augmente de 1.9948 unités, ce qui indique un impact très significatif.

**Expenditure (Dépenses)** : 0.0648
Chaque unité d'augmentation des dépenses entraîne une augmentation du PIB de 0.0648 unités, montrant un impact positif et relativement faible.

En somme, bien que les revenus , la composante de demende aggrée et  la production ont un impact beaucoup plus marqué, toutes les variables notament (les dépenses gouvernementales,les corporations et l'investissement en capital) ont des effets positifs et significatifs sur le PIB ,chacune à des degrés différents.

**Households (Ménages)** : -0.3136
Une augmentation des variables associées aux ménages est liée à une diminution du PIB de 0.3136 unités, suggérant un effet négatif.



#### Étape 11: Comparaison entre la Régression  Classique et Ridge

In [230]:

# Comparaison des coefficients
comparison = pd.concat([coefficients_linear, coefficients_ridge], axis=1)
comparison.columns = ['Régression Classique', 'Régression Ridge']
print("Comparaison des Coefficients:")
print(comparison)

Comparaison des Coefficients:
                                Régression Classique  Régression Ridge
Income                                      0.099650          0.093455
Components of aggregate demand              0.262501          0.192340
Expenditure                                -0.053378          0.064755
Production                                  2.095101          1.994780
Households                                 -0.328577         -0.313627
Government                                  0.046905          0.051668
Corporations                                0.135829          0.114302
Capital                                     0.109041          0.125254
Government expenditure by fun               0.046905          0.051668


Dans l'ensemble,les deux modèles montrent des impacts similaires pour la plupart des variables, la différence marquée pour les dépenses'Expenditure' souligne l'importance de la régularisation dans la régression Ridge. 

#### Explication de la différence marquée pour les dépenses

**Sans Régularisation (Régression Classique)**

-Dans le modèle classique, le coefficient des dépenses est négatif (-0.053378), suggérant qu'une augmentation des dépenses est associée à une légère diminution du PIB. Ce résultat peut donner l'impression que les dépenses sont nuisibles pour l'économie.

-Cependant, ce résultat peut être influencé par la **Multicolinéarité**, où des corrélations élevées entre les dépenses et d'autres variables comme le revenu ou la production complique l'évaluation de l'impact direct des dépenses sur le PIB. 
 
-Scénario : Supposons que le gouvernement augmente les dépenses pour un projet d'infrastructure. Cela entraîne une hausse de la production (les entreprises sont engagées) et du revenu (les travailleurs sont payés).

-Dans ce cas, le modèle classique pourrait attribuer une partie de l’augmentation du PIB à la production ou au revenu, masquant ainsi l'effet réel des dépenses elles-mêmes sur le PIB.

**Avec Régularisation (Régression Ridge)**

-Dans le modèle Ridge, le coefficient devient positif  (0.064755), indiquant qu'une augmentation des dépenses  entraîne une augmentation du PIB de 0.0648 unités, montrant un impact positif mais relativement faible.

-En appliquant une pénalité aux coefficients, la régression Ridge ajuste les effets des variables corrélées. Cela permet de mieux estimer l'effet direct des dépenses sur le PIB, indépendamment des effets confondants des autres variables (comme la production et le revenu).

-Scénario Révisé :lorsque le gouvernement augmente les dépenses vers des projets d'infrastructure, le modèle peut estimer que l'effet positif sur le PIB provient en grande partie de ces dépenses, même si la production et le revenu augmentent également.Ce qui n’était pas visible dans le modèle classique.


#### Résumé

En résumé, le passage d'un coefficient négatif à un coefficient positif pour  les dépenses "Expenditure" entre les deux modèles met en évidence les limitations de la régression classique, notamment face à la multicolinéarité  et souligne l'importance de la régularisation dans l'analyse des données économiques. Sans régularisation, les effets des dépenses peuvent être masqués par la multicolinéarité .
s.

Avec la régression Ridge, les interactions complexes entre les variables sont mieux prises en com en réduisant l'impact des multicolinéarités rendant ainsi nos résultats plus fiables en cas de corrélations élevées entre les variables,e, permettant une évaluation plus précise de l'impact des dépenses sur le PIB.